In [1]:
!pip3 install pyarrow pandas sklearn

    100% |████████████████████████████████| 11.6MB 120kB/s 
    100% |████████████████████████████████| 8.7MB 157kB/s 
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 13.8MB 100kB/s 
  Using cached https://files.pythonhosted.org/packages/74/68/d87d9b36af36f44254a8d512cbfc48369103a3b9e474be9bdfe536abfc45/python_dateutil-2.7.5-py2.py3-none-any.whl
    100% |████████████████████████████████| 512kB 2.8MB/s 
    100% |████████████████████████████████| 5.3MB 269kB/s 
    100% |████████████████████████████████| 26.5MB 47kB/s 
  Running setup.py bdist_wheel for sklearn ... - done
  Stored in directory: /home/Hoiy/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [0]:
!sudo apt-get install unzip
!wget http://sighan.cs.uchicago.edu/bakeoff2005/data/icwb2-data.zip
!unzip icwb2-data.zip

In [0]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

In [0]:
import os
import tokenization
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# from google.colab import auth
# auth.authenticate_user()

In [0]:
BERT_PRETRAINED_DIR = 'gs://dev-test-bert-tpu/chinese_bert/'
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
BERT_MODEL = 'chinese_L-12_H-768_A-12'
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)

In [6]:
data = pd.read_csv('./icwb2-data/training/pku_training.utf8', header=None)
display(data.head())

train, test = train_test_split(data, test_size=0.2, shuffle=False)
train.shape, test.shape

,0
0,迈向 充满 希望 的 新 世纪 —— 一九九八年 新年 讲话 （ 附 ...
1,中共中央 总书记 、 国家 主席 江 泽民
2,（ 一九九七年 十二月 三十一日 ）
3,１２月 ３１日 ， 中共中央 总书记 、 国家 主席 江 泽民 发表 ...
4,同胞 们 、 朋友 们 、 女士 们 、 先生 们 ：


((15243, 1), (3811, 1))

In [7]:
class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None, label=None):
    """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label

      
def get_train_examples():
  examples = []
  
  def append_example(row):   
    examples.append(InputExample(row.name, row[0]))

  train.apply(append_example, axis=1)
  return examples


def get_test_examples():
  examples = []
  def append_example(row):
    examples.append(InputExample(row.name, row[0]))
  
  test.apply(append_example, axis=1)
  return examples


train_examples = get_train_examples()
test_examples = get_test_examples()
train_examples[:3], len(train_examples)

([<__main__.InputExample at 0x7fd84f44bc50>,
 15243)

In [8]:
l = list(filter(None, train_examples[5].text_a.split(' ')))
print(''.join(l))
t = ['１' if i < len(token) - 1 else '在' for token in l for i in range(len(token))]
print(''.join(t))

在１９９８年来临之际，我十分高兴地通过中央人民广播电台、中国国际广播电台和中央电视台，向全国各族人民，向香港特别行政区同胞、澳门和台湾同胞、海外侨胞，向世界各国的朋友们，致以诚挚的问候和良好的祝愿！
在１１１１在１在１在在在１在１在在１在１在１在１在１在在１在１在１在１在在１在１１在在在１在１在１在在在１在１在１１在１在在１在在１在１在在１在１在在在１在１在在１在在在１在１在在１在在１在在１在在


In [0]:
# # For char count frequency
# keras_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, oov_token='<oov>', split='')
# keras_tokenizer.fit_on_texts(train.main_content)
# len(keras_tokenizer.index_word), keras_tokenizer.word_index['<oov>']

# choice = []
# p = []

# for k, v in keras_tokenizer.word_counts.items():
#   if k in tokenizer.vocab:
#     choice.append(k)
#     p.append(v)

# p = p / np.sum(p)

# choice[:10], p[:10]

In [9]:
class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               truths,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.is_real_example = is_real_example
    self.truths = truths

    
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
  """Converts a single `InputExample` into a single `InputFeatures`."""
  
  text = list(filter(None, example.text_a.split(' ')))
  tokens_a_truth = [0. if i < len(token) - 1 else 1. for token in text for i in range(len(token))]
  text_a = ''.join(text)
    
  assert len(text_a) == len(tokens_a_truth)
  
  tokens_a = tokenizer.tokenize(text_a)
  tokens_b = None
  if example.text_b:
    tokens_b = tokenizer.tokenize(example.text_b)

  if tokens_b:
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
  else:
    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[0:(max_seq_length - 2)]

  # The convention in BERT is:
  # (a) For sequence pairs:
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  tokens = []
  segment_ids = []
  truths = []
  
  tokens.append("[CLS]")
  segment_ids.append(0)
  truths.append(1.)
  
  for token, truth in zip(tokens_a, tokens_a_truth):
    tokens.append(token)
    segment_ids.append(0)
    truths.append(truth)
    
  tokens.append("[SEP]")
  segment_ids.append(0)
  truths.append(1.)

  if tokens_b:
    for token in tokens_b:
      tokens.append(token)
      segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)
    truths.append(1.)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)
    truths.append(0.)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length
  assert len(truths) == max_seq_length

  if ex_index < 3:
    tf.logging.info("*** Example ***")
    tf.logging.info("guid: %s" % (example.guid))
    tf.logging.info("tokens: %s" % " ".join(
        [tokenization.printable_text(x) for x in tokens]))
    tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
    tf.logging.info("truths: %s" % " ".join([str(x) for x in truths]))

  feature = InputFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      truths=truths
  )
  return feature


def convert_examples_to_features(examples, max_seq_length,
                                 tokenizer):
  """Convert a set of `InputExample`s to a list of `InputFeatures`."""

  features = []
  for (ex_index, example) in enumerate(examples):
    if ex_index % 1000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example,
                                     max_seq_length, tokenizer)

    features.append(feature)
  return features  

print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')

MAX_SEQ_LENGTH = 128 #@param {'type': 'number'}

train_features = convert_examples_to_features(train_examples, MAX_SEQ_LENGTH, tokenizer)
test_features = convert_examples_to_features(test_examples, MAX_SEQ_LENGTH, tokenizer)
train_features[:10], len(train_features)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...
INFO:tensorflow:Writing example 0 of 15243
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 0
INFO:tensorflow:tokens: [CLS] 迈 向 充 满 希 望 的 新 世 纪 [UNK] [UNK] 一 九 九 八 年 新 年 讲 话 （ 附 图 片 １ 张 ） [SEP]
INFO:tensorflow:input_ids: 101 6815 1403 1041 4007 2361 3307 4638 3173 686 5279 100 100 671 736 736 1061 2399 3173 2399 6382 6413 8020 7353 1745 4275 8029 2476 8021 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

([<__main__.InputFeatures at 0x7fd84f181d68>,
 15243)

In [0]:
!mkdir -p dataset

import pickle

with open('./dataset/train_features_%s.pkl'%MAX_SEQ_LENGTH, 'wb') as f:
  pickle.dump(train_features, f)

  
with open('./dataset/test_features_%s.pkl'%MAX_SEQ_LENGTH, 'wb') as f:
  pickle.dump(test_features, f)

# !gsutil rsync ./dataset gs://dev-test-bert-tpu/dataset